In [93]:
%matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t
from sklearn.linear_model import LinearRegression

Using matplotlib backend: Qt5Agg


# Question 1


In [37]:
fields=["year","gnp","invest","cpi","interest"]
invest = pd.read_csv("invest.txt",sep=' ',usecols=fields)

In [38]:
invest.head()

,year,gnp,invest,cpi,interest
0,1968,873.4,133.3,82.54,5.16
1,1969,944.0,149.3,86.79,5.87
2,1970,992.7,144.2,91.45,5.95
3,1971,1077.6,166.4,96.01,4.88
4,1972,1185.9,195.0,100.00,4.50


## Question 2

In [41]:
x=np.log(invest["gnp"])
y=np.log(invest["invest"])
plt.plot(x,y)
plt.show()

## Question 3


In [55]:
X=invest.gnp.describe().aggregate(np.mean)
Y=invest.invest.describe().aggregate(np.mean)


n=len(invest)
s=0
s1=0
for i in range(n):
    s+=(invest["gnp"][i]-X)*(invest["invest"][i]-Y)
    s1+=(invest["gnp"][i]-X)**2
beta_1=s/s1
beta_0=Y-beta_1*X
print("beta_0=",beta_0)
print("beta_1=",beta_1)

beta_0= 7.3982674947376665
beta_1= 0.15425089907838888


## Question 4

In [56]:
temp=0
for i in range(n):
    temp+=(invest["invest"][i]-(beta_0+beta_1*invest["gnp"][i]))**2
sigma=temp/(n-2)
var_b0=sigma*((1/n)+((X)**2)/s1)
var_b1=sigma/s1
print("Var(beta_0)=",var_b0)
print("Var(beta_1)=",var_b1)

Var(beta_0)= 230.27575704386697
Var(beta_1)= 8.705600423618703e-05


In [74]:
random_values=t.rvs(n-2,  size = 1000)
quantiles = np.quantile(random_values,0.55)
print("[-",quantiles,",",quantiles,"]")
print((beta_0/var_b0),-quantiles<=(beta_0/var_b0)<=quantiles)
print((beta_1/var_b1),-quantiles<=(beta_1/var_b1)<=quantiles)

[- 0.08105189666336302 , 0.08105189666336302 ]
0.03212786091645902 True
1771.8582472485061 False


The p-value is

In [75]:
1-t.cdf(Y,0.97)

0.0016389941804687558

## Question 5

In [76]:
inv=beta_0+beta_1*1000
print("The estimated necessary Investment for a GNP of 1000 is ",inv)

The estimated necessary Investment for a GNP of 1000 is  161.64916657312654


In [82]:
def CI(x_input):
    c0=beta_0+beta_1*x_input-quantiles*np.sqrt(sigma)*np.sqrt((1/n)+((x_input-X)**2)/s1)
    c1=beta_0+beta_1*x_input+quantiles*np.sqrt(sigma)*np.sqrt((1/n)+((x_input-X)**2)/s1)
    return (c0,c1)
def PI(x_input):
    p0=beta_0+beta_1*x_input-quantiles*np.sqrt(sigma)*np.sqrt(1+(1/n)+((x_input-X)**2)/s1)
    p1=beta_0+beta_1*x_input+quantiles*np.sqrt(sigma)*np.sqrt(1+(1/n)+((x_input-X)**2)/s1)
    return (p0,p1)
print("confidence interval : ",CI(1000)[0],CI(1000)[1])    
print("prediction interval :", PI(1000)[0],PI(1000)[1])

confidence interval :  160.97529243282224 162.32304071343083
prediction interval : 159.2591676545533 164.03916549169978


## Question 6

In [91]:
x=np.log(invest["gnp"])
y=np.log(invest["invest"])
plt.plot(x,y,label="data")
z=[np.log(beta_0+beta_1*invest["gnp"][i]) for i in range(n)]
plt.plot(x,z,label="estimated regression line")
plt.legend()
for i in range(n):
    plt.plot((x[i],x[i]),(np.log(CI(invest["gnp"][i])[0]),np.log(CI(invest["gnp"][i])[1])),'ro-',color='red')
    plt.plot((x[i],x[i]),(np.log(PI(invest["gnp"][i])[0]),np.log(PI(invest["gnp"][i])[1])),'ro-',color='green')
plt.show

<function matplotlib.pyplot.show(*args, **kw)>

## Question 7

In [99]:
X0=[]
Y0=[]
for i in range(n):
    X0.append([invest["gnp"][i]])
    Y0.append(invest["invest"][i])
reg=LinearRegression().fit(X0,Y0)
print("slope =",reg.coef_,"intercept =", reg.intercept_,"determination coefficient =",reg.score(X0,Y0))
reg.predict(1000)

slope = [0.15496368] intercept = 5.02993790597327 determination coefficient = 0.946362204399403


array([159.99362149])

## Question 8

In [107]:
x=np.log(invest["gnp"])
y=np.log(invest["invest"])
plt.plot(x,y,label="data")
z=[np.log(reg.intercept_+reg.coef_*invest["gnp"][i]) for i in range(n)]
plt.plot(x,z,label="estimated regression line")
temp=[np.log(reg.predict(invest["gnp"][i])) for i in range(n)]
for i in range(n):
    plt.plot(x,temp,color="green")
plt.legend()
plt.show()

## Question 9